In [51]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
from statsmodels.graphics.gofplots import qqplot
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import sklearn.metrics
%matplotlib inline

In [52]:
titanic_df = pd.read_csv('data/titanic.csv')
titanic_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [53]:
titanic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [54]:
# pid can be removed
# pclass - good
# sex - dummies
y = titanic_df.Survived
titanic_df['is_female'] = pd.get_dummies(titanic_df.Sex)['female']
X = titanic_df.drop(columns=['PassengerId', 'Survived', 'Name', 'Sex'])
X.head()

,Pclass,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,is_female
0,3,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,1,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,1,35.0,1,0,113803,53.1000,C123,S,1
4,3,35.0,0,0,373450,8.0500,NaN,S,0


In [55]:
#pd.cut(titanic_df.Age, 3)
X['ager'] = pd.cut(titanic_df.Age, 3, labels=[1, 2, 3])
X = X.drop(columns=['Age'])
X.head()

,Pclass,SibSp,Parch,Ticket,Fare,Cabin,Embarked,is_female,ager
0,3,1,0,A/5 21171,7.2500,NaN,S,0,1
1,1,1,0,PC 17599,71.2833,C85,C,1,2
2,3,0,0,STON/O2. 3101282,7.9250,NaN,S,1,1
3,1,1,0,113803,53.1000,C123,S,1,2
4,3,0,0,373450,8.0500,NaN,S,0,2


In [56]:
X['fam_size'] = X.SibSp + X.Parch
X = X.drop(columns=['SibSp', 'Parch'])

In [57]:
X.Embarked.value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [58]:
X['Embarked'] = pd.Categorical(X.Embarked, ordered=True).codes
X.head()

,Pclass,Ticket,Fare,Cabin,Embarked,is_female,ager,fam_size
0,3,A/5 21171,7.2500,NaN,2,0,1,1
1,1,PC 17599,71.2833,C85,0,1,2,1
2,3,STON/O2. 3101282,7.9250,NaN,2,1,1,0
3,1,113803,53.1000,C123,2,1,2,1
4,3,373450,8.0500,NaN,2,0,2,0


In [59]:
X = X.drop(columns=['Ticket'])
X.head()

,Pclass,Fare,Cabin,Embarked,is_female,ager,fam_size
0,3,7.2500,NaN,2,0,1,1
1,1,71.2833,C85,0,1,2,1
2,3,7.9250,NaN,2,1,1,0
3,1,53.1000,C123,2,1,2,1
4,3,8.0500,NaN,2,0,2,0


In [60]:
# NaN is considered as float
#X.Cabin.map(lambda c: c)
X['Cabin'] = X.Cabin.map(lambda c: 0 if pd.isna(c) else ord(c[0]))
X.head()

,Pclass,Fare,Cabin,Embarked,is_female,ager,fam_size
0,3,7.2500,0,2,0,1,1
1,1,71.2833,67,0,1,2,1
2,3,7.9250,0,2,1,1,0
3,1,53.1000,67,2,1,2,1
4,3,8.0500,0,2,0,2,0


In [75]:
roll1 = np.random.randint(1, 7, 1000)
roll2 = np.random.randint(1, 7, 1000)
result = roll1+roll2
sum((result > 7) or (result % 2 == 0))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [83]:
sum((result > 7)) + sum((result < 7) & (result % 2 == 0)) / 2

556.0

In [80]:
(result < 7) & (result % 2 == 0)

array([False, False, False, False, False, False,  True, False,  True,
       False,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True,  True,  True, False, False, False, False, False,
       False, False, False, False, False,  True, False, False, False,
        True,  True, False,  True, False, False, False, False,  True,
       False, False, False, False,  True, False,  True, False,  True,
       False, False,  True,  True, False,  True,  True, False,  True,
       False, False, False, False, False, False,  True, False,  True,
        True,  True, False, False, False, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False,  True, False,  True, False, False,  True,
       False,  True, False, False, False, False, False, False,  True,
       False, False, False,  True, False, False, False,  True, False,
        True, False,